In [79]:
%reset
%matplotlib qt5

In [80]:
# Library imports

import mne
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os.path
import glob
from scipy import signal
from scipy.signal import savgol_filter, lfilter
from numpy.fft import fft, fftfreq
from mne.filter import filter_data
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs, create_eog_epochs)
from mne.time_frequency.tfr import morlet
from mne.io import read_raw_brainvision as bvread
from mne.viz import plot_filter, plot_ideal_filter
from ipywidgets import interact, fixed
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from matplotlib.animation import FuncAnimation

In [81]:
# Function to save dynamic figures with a scrollbar
# TODO: This function currently does not work as expected. It requires debugging.

import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def create_figure(data, title):
    # Create a tkinter window
    root = tk.Tk()
    root.title(title)

    # Create a figure object
    fig = plt.figure()

    # Create a canvas to hold the figure
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    # Create a scrollbar to control the time range
    scrollbar = tk.Scale(root, from_=0, to=data.times[-1], orient=tk.HORIZONTAL, length=500, resolution=0.1)
    scrollbar.pack(side=tk.BOTTOM, fill=tk.X)

    # Create a function to update the plot when the scrollbar is moved
    def update_plot(val, fig):
        # Get the current position of the scrollbar
        pos = scrollbar.get()

        # Update the figure with the new time range
        fig.clear()
        data.plot(start=pos, duration=1, show=False)
        canvas.draw()

    # Set the initial position of the scrollbar to 0
    scrollbar.set(0)

    # Call the update function to plot the initial data
    update_plot(0, fig)

    # Bind the update function to the scrollbar
    scrollbar.config(command=lambda val: update_plot(val, fig))

    # Start the tkinter event loop
    root.mainloop()

    # Return the figure object with the current time range
    return fig



In [82]:
# Verification of files: are all necessary files present? Necessary files: .vhdr for each block/session/participant

import os.path

MISSING_FILES = []
START = 1
END = 2
INTERVAL = 1
ALL_SUB = list(range(START, END +1, INTERVAL))

for SUBJECT in ALL_SUB:

    SES_NUM = [1,2]
    for SESSION in SES_NUM :
        
        BLOCK_NUMBER = [1,2,3,4,5,6]#,7]
        for BLOCK in BLOCK_NUMBER:
            
            PATH = '/home/lenouvju-admin/Bureau/BIDS_linux/derivatives/Pipeline2_GazeEegSynchro/sub-{sub:02d}/ses-{ses:02d}/B{blk}/synchro_P{sub:02d}{ses:02d}_B{blk}.asc.vhdr'.format(sub= SUBJECT,ses = SESSION, blk = BLOCK)
            isFile = os.path.isfile(PATH)
            if isFile == False:
                MISSING_FILES.append(PATH)
                print('File does not exist: ' + PATH)

if len(MISSING_FILES) > 0 :
    print ( str(len(MISSING_FILES)) + ' files are missing. \nPlease check that all .vhdr files exist and are in the correct path before continuing.')


In [83]:
# Setting the range of values for SUB list using START, END, and INTERVAL
START = 1
END = 2
INTERVAL = 1
SUB = list(range(START, END +1, INTERVAL))

# Defining the list of session numbers for each subject
SES_NUM = [1,2] #,2]

# Threshold value for correlation of match between ICA pattern and EOGV-EOGH channels
# TODO: Find the optimal threshold value from literature
threshold_ICA = 0.6 

# Looping through all subjects in the specified range
for SUBJECT in ALL_SUB:
    
    # Looping through all sessions for each subject
    for SESSION in SES_NUM:
        # Creating an empty dataframe to store the results for each subject and session
        Dataframe_P_S_B = pd.DataFrame()

        # Defining the path to save the figures generated by the pipeline
        BEGIN_PATH_SAVE_FIGURE = '/home/lenouvju-admin/Bureau/BIDS_linux/results/Result_Pipeline6_Pretreatment_EEG/sub-{sub:02d}/ses-{ses:02d}/'.format(sub= SUBJECT,ses = SESSION)

        # Looping through all blocks for each subject and session
        BLOCK_NUMBER = [1,2,3,4,5,6]#,7]
        for BLOCK in BLOCK_NUMBER:
            # Defining the paths for EEG data and corresponding events for each block
            PATH = '/home/lenouvju-admin/Bureau/BIDS_linux/derivatives/Pipeline2_GazeEegSynchro/sub-{sub:02d}/ses-{ses:02d}/B{blk}/synchro_P{sub:02d}{ses:02d}_B{blk}.asc.vhdr'.format(sub= SUBJECT,ses = SESSION, blk = BLOCK)
            EVENTS_PATH = '/home/lenouvju-admin/Bureau/BIDS_linux/derivatives/Pipeline2_GazeEegSynchro/sub-{sub:02d}/ses-{ses:02d}/B{blk}/synchro_P{sub:02d}{ses:02d}_B{blk}.asc.vmrk'.format(sub= SUBJECT,ses = SESSION, blk = BLOCK)
            # Check if the file exists
            isFile = os.path.isfile(PATH)

            # Define the participant ID based on subject, session, and block
            Participant = 'P{sub:02d}{ses:02d}_B{blk}'.format(sub=SUBJECT, ses=SESSION, blk=BLOCK)

            # Define the channels to read from the raw file
            eog_channels = ["EOGV", "EOGH"]
            ecg_channels = ["ECG"]
            misc_channels = ["eyeRx", "eyeRy", "blink"] # Note: depending on the participant, "eyeRx" may be "eyeLx" if the dominant eye is the left one
            raw = mne.io.read_raw_brainvision(PATH, eog=[eog_channels + ecg_channels], misc=misc_channels, preload=True)

            # Set the channel types
            raw.set_channel_types({ch: 'eog' for ch in eog_channels})
            raw.set_channel_types({ch: 'ecg' for ch in ecg_channels})
            raw.set_channel_types({ch: 'misc' for ch in misc_channels})

            # Initialize a dictionary to store the results for this block
            Dict_P_S_B = {'P_S_B': Participant, 'Bad_channels_before_correction': [], 'Sources_ICA_rejected': []}
            # P_S_B stands for Participant_Session_Block.
            # The final dataframe will have a row for each participant/session/block/epoch combination

            # Check that no projectors are already present in the raw fif file
            if raw.info['projs']:
                raise (ValueError)

            # Set the montage
            montage = mne.channels.make_standard_montage('easycap-M1') # TODO: Consider using M10 instead of M1
            raw.set_montage(montage, on_missing='warn', verbose=False)

            # Store a copy of the raw data before removing bad channels
            RAW_BEFORE_BAD_CHANNELS_REMOVED = raw.copy()
            print('Bads_channels_detected: ', RAW_BEFORE_BAD_CHANNELS_REMOVED.info['bads'])

            # Store the bad channels that were detected before any removing was applied
            Dict_P_S_B['Bad_channels_before_correction'] = RAW_BEFORE_BAD_CHANNELS_REMOVED.info['bads']

            # Remove bad channels, if any
            raw = raw.drop_channels(RAW_BEFORE_BAD_CHANNELS_REMOVED.info['bads'])

            # Pick EEG and EOG channels
            eeg_eog_picks = mne.pick_types(raw.info, misc=False, ecg=False, eeg=True, eog=True, stim=False)
            eeg_picks = mne.pick_types(raw.info, misc=False, ecg=False, eeg=True, eog=False, stim=False)
            eog_picks = mne.pick_types(raw.info, misc=False, ecg=False, eeg=False, eog=True, stim=False)

            # Define the EOG channel names
            eog_channel_names = ['EOGV', 'EOGH']

            # Pass band filter
            raw_F= raw.copy()
            l_p = 150.0  # low-pass filter to suppress noise above 150 Hz
            h_p = 0.2

            raw_Filtered = raw_F.filter(l_freq=h_p, h_freq=l_p, fir_design='firwin', picks=eeg_eog_picks, filter_length='auto', method='fir', phase='zero', verbose=True)

            notches = [50, 85]
            raw_Filtered = raw_Filtered.notch_filter(notches, phase='zero-double', fir_design='firwin')

            # Apply low-frequency filter for ICA
            filt_raw_pour_ICA = raw_Filtered.copy().filter(l_freq=1.0, h_freq=None)

            # TODO : @Ronald, @Steeve, @Alan : The warning message "you should use data filter .filter(l_freq=1.0, h_freq=None) " is still shown even when I do it ! Is it because I am using the derivative ?

            # Define the filter parameters
            fs = filt_raw_pour_ICA.info['sfreq']  # sampling frequency
            f0 = fs / 4  # center frequency of the filter, in Hz
            # sigma value: Should have a slope consistent with speed. Average saccade time is 131 ms for parts 12 to 17.
            # TODO: Should I take the average saccade duration per participant or in general? Ronald's response: in general. @Steeve and @Alan responses ?
            sigma =  0.0131#(f0 / (2 * np.pi)) / 1000 # f0 / (2 * np.pi)  # standard deviation of the filter, in seconds
            n = 1/10 *fs #int(sigma * fs * 8) + 1  # length of the filter kernel (must be odd)

            # Create the filter kernel (a Gaussian function)
            t = np.arange(n) / fs - (n - 1) / (2 * fs)
            kernel = signal.gaussian(n, sigma * fs)

            kernel = np.diff(kernel)
            # Normalize the kernel to have unit energy
            kernel /= np.sqrt(np.sum(kernel ** 2))

            # Apply the derivative filter to the data
            raw_deriv = filt_raw_pour_ICA.copy()
            raw_deriv = raw_deriv.filter(l_freq=None, h_freq=None, method='fir', fir_window='hamming', fir_design='firwin', phase='zero-double', verbose='INFO')
            raw_deriv._data = signal.convolve(raw_deriv._data, kernel[np.newaxis, :], mode='valid')

            # Create a new Raw instance of the derivative data
            ch_types_Ronald = [mne.io.pick.channel_type(filt_raw_pour_ICA.info, n) for n in range(70)]
            info_filt = mne.create_info(ch_names=filt_raw_pour_ICA.ch_names, sfreq=filt_raw_pour_ICA.info['sfreq'], ch_types=ch_types_Ronald)
            raw_deriv = mne.io.RawArray(raw_deriv._data, info_filt)
            raw_deriv.set_montage(montage, on_missing='warn', verbose=False)

            # Fitting ICA on derivative data to remove EOG artifacts
            ica_pour_derivative = mne.preprocessing.ICA(n_components=0.90, method='picard', fit_params=dict(extended=False), max_iter='auto', random_state=97)  # TODO: check the purpose of fit_params
            ica_pour_derivative.fit(raw_deriv, picks = eeg_picks)  # fit ICA on derivative data to better identify EOG artifacts, TODO: fit on each epoch? No each trial ?
            # Consider applying another filter at the epoch level to avoid edge effects

            ica_pour_derivative.exclude = []
            # Identify ICs with EOG artifacts
            eog_indices, eog_scores = ica_pour_derivative.find_bads_eog(raw_deriv, threshold =0.7, measure = "correlation")
            ica_pour_derivative.exclude = eog_indices

            # Apply ICA to the raw data (not derivative)
            raw_after_ICA = raw_Filtered.copy()
            ica_pour_derivative.apply(raw_after_ICA)

            # Save the filtered data in BrainVision format
            path_raw_filtered = BEGIN_PATH_SAVE_FIGURE + Participant + '_RAW_AFTER_ICA.fif'
            raw_after_ICA.save(path_raw_filtered, overwrite=True)

            # Plot 

            # plot RAW Filtered before ICA
            raw_Filtered_plot= raw_Filtered.plot()
            plt.subplots_adjust(top=0.9)
            raw_Filtered_plot.suptitle("Raw filtered before ICA. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_1_RAW_FILTERED_BEFORE_ICA'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot the filter apply to the raw_data
            filter_apply_plot = mne.viz.plot_filter(kernel, filt_raw_pour_ICA.info['sfreq'])
            plt.subplots_adjust(top=0.9)
            filter_apply_plot.suptitle("Filter apply on Raw data to derive it. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_2_FILTER_APPLIED'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot derivative of raw data before ICA application
            raw_deriv_plot = raw_deriv.plot()
            plt.subplots_adjust(top=0.9)
            raw_deriv_plot.suptitle("Derivative of Raw data before ICA application. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_3_RAW_DERIVATIVE'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # Plot the ICA sources
            source_ica_plot = ica_pour_derivative.plot_sources(raw_deriv, show_scrollbars=True, start=0, stop=20)
            plt.subplots_adjust(top=0.9)
            source_ica_plot.suptitle("Composantes ICA fig_ica_pour_derivative_EEG_channels. "+ Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_4_ICA_DERIVATIVE_EEG'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # barplot of ICA component "EOG match" scores
            EOG_match_score_plot = ica_pour_derivative.plot_scores(eog_scores)
            plt.subplots_adjust(top=0.9)
            EOG_match_score_plot.suptitle("EOG Match scores. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_5_ EOG_MATCH_SCORES'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot diagnostics
            diagnostics_plot = ica_pour_derivative.plot_properties(raw_deriv, picks=eog_indices)
            plt.subplots_adjust(top=0.9)
            figD = plt.gcf()
            figD.suptitle("Diagnostics")
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_6_ICA_DIAGNOSTICS'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot ICs applied to raw data, with EOG matches highlighted
            ICs_plot = ica_pour_derivative.plot_sources(raw_deriv, show_scrollbars=False)
            plt.subplots_adjust(top=0.9)
            ICs_plot.suptitle("ICs. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_7_ICs'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot RAW AFTER ICA
            raw_after_ICA_plot= raw_after_ICA.plot()
            plt.subplots_adjust(top=0.9)
            raw_after_ICA_plot.suptitle("Raw filtered after ICA application. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_8_RAW_AFTER_ICA'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

Extracting parameters from /home/lenouvju-admin/Bureau/BIDS_linux/derivatives/Pipeline2_GazeEegSynchro/sub-01/ses-01/B1/synchro_P0101_B1.asc.vhdr...
Setting channel info structure...
Reading 0 ... 446472  =      0.000 ...   446.472 secs...


/tmp/ipykernel_14652/205947883.py:41: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(PATH, eog=[eog_channels + ecg_channels], misc=misc_channels, preload=True)
/tmp/ipykernel_14652/205947883.py:41: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(PATH, eog=[eog_channels + ecg_channels], misc=misc_channels, preload=True)


Bads_channels_detected:  []
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.2 - 1.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Upper passband edge: 150.00 Hz
- Upper transition bandwidth: 37.50 Hz (-6 dB cutoff frequency: 168.75 Hz)
- Filter length: 16501 samples (16.501 sec)



/tmp/ipykernel_14652/205947883.py:46: RuntimeWarning: The unit for channel(s) blink, eyeRx, eyeRy has changed from NA to V.
  raw.set_channel_types({ch: 'misc' for ch in misc_channels})
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 sec)



...

Filtering raw data in 1 contiguous segment

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal allpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Filter length: 1 samples (0.001 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.9s finished


Creating RawArray with float64 data, n_channels=70, n_times=428054
    Range : 0 ... 428053 =      0.000 ...   428.053 secs
Ready.
Fitting ICA to data using 64 channels (please be patient, this may take a while)


/tmp/ipykernel_14652/205947883.py:123: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica_pour_derivative.fit(raw_deriv, picks = eeg_picks)  # fit ICA on derivative data to better identify EOG artifacts, TODO: fit on each epoch? No each trial ?
/home/lenouvju-admin/anaconda3/envs/mne/lib/python3.10/site-packages/mne/preprocessing/ica.py:774: RuntimeWarning: invalid value encountered in divide
  data /= self.pre_whitener_


ValueError: array must not contain infs or NaNs

In [ ]:
# Code avant correction des commentaires

'''
# TODO: Remove segments that are not trial data (not between 20 - 29)

START = 1
END = 1
INTERVAL = 1
SUB = list(range(START, END +1, INTERVAL))
SES_NUM = [1]#,2]

# Treshold value pour corrrelation de match entre pattern ICA et canaux EOGV-EOGH
threshold_ICA = 0.6 # TODO voir dans la littérature à combien est mis le treshold

for SUBJECT in ALL_SUB:
    # Loop through all subjects in the specified range

    for SESSION in SES_NUM :
        # Loop through all sessions for each subject
        Dataframe_P_S_B = pd.DataFrame()

        BEGIN_PATH_SAVE_FIGURE = '/home/lenouvju-admin/Bureau/BIDS_linux/results/Result_Pipeline6_Pretreatment_EEG/sub-{sub:02d}/ses-{ses:02d}/'.format(sub= SUBJECT,ses = SESSION)

        BLOCK_NUMBER =[1]#,2,3,4,5,6,7]
        for BLOCK in BLOCK_NUMBER:
            # Loop through all blocks for each subject and session

            PATH = '/home/lenouvju-admin/Bureau/BIDS_linux/derivatives/Pipeline2_GazeEegSynchro/sub-{sub:02d}/ses-{ses:02d}/B{blk}/synchro_P{sub:02d}{ses:02d}_B{blk}.asc.vhdr'.format(sub= SUBJECT,ses = SESSION, blk = BLOCK)
            EVENTS_PATH = '/home/lenouvju-admin/Bureau/BIDS_linux/derivatives/Pipeline2_GazeEegSynchro/sub-{sub:02d}/ses-{ses:02d}/B{blk}/synchro_P{sub:02d}{ses:02d}_B{blk}.asc.vmrk'.format(sub= SUBJECT,ses = SESSION, blk = BLOCK)
            
            isFile = os.path.isfile(PATH)

            Participant = 'P{sub:02d}{ses:02d}_B{blk}'.format(sub = SUBJECT, ses = SESSION, blk = BLOCK)
            
            eog_channels = ["EOGV", "EOGH"]
            ecg_channels = ["ECG"]
            misc_channels = ["eyeRx", "eyeRy", "blink"]# Attention selon le  participant, c'est eyeRx ou eyeLx selon si l'oeil directeur est droit ou gauche (left). Inplémenter plus tard un truc qui permetde prendr een compte ces deux cas de figures
            raw = mne.io.read_raw_brainvision(PATH, eog=[eog_channels + ecg_channels], misc=misc_channels, preload=True)

            # Set the channel types
            raw.set_channel_types({ch: 'eog' for ch in eog_channels})
            raw.set_channel_types({ch: 'ecg' for ch in ecg_channels})
            raw.set_channel_types({ch: 'misc' for ch in misc_channels})
            
 
            # Load the raw data from a BrainVision file for each block of each subject and session

            Dict_P_S_B = {'P_S_B': Participant , 'Bad_channels_before_correction': [], 'Sources_ICA_rejected': []}
            # Initialize a dictionary to store the results for this block
            # P_S_B stands for Participant_Session_Block. 
            # The final dataframe will have a row for each participant/session/block/epoch combination

            if raw.info['projs']:
                raise (ValueError)
            # Check that no projectors are already present in the raw fif file

            # Set montage
            montage = mne.channels.make_standard_montage('easycap-M1') # TODO : Hésitation between M1 and M10, à vérifier au CHU
            raw.set_montage(montage, on_missing = 'warn', verbose=False)

            RAW_BEFORE_BAD_CHANNELS_REMOVED = raw.copy()
            print('Bads_channels_detected : ', RAW_BEFORE_BAD_CHANNELS_REMOVED.info['bads'])
            
            Dict_P_S_B['Bad_channels_before_correction'] = RAW_BEFORE_BAD_CHANNELS_REMOVED.info['bads']
            # Store the bad channels that were detected before any removing was applied

            # Remove bad channels, if any
            raw = raw.drop_channels(RAW_BEFORE_BAD_CHANNELS_REMOVED.info['bads'])

            eeg_eog_picks = mne.pick_types(raw.info, misc=False, ecg= False, eeg=True, eog=True, stim=False)


            eeg_picks = mne.pick_types(raw.info, misc=False, ecg= False, eeg=True, eog=False, stim=False)
            eog_picks = mne.pick_types(raw.info, misc=False, ecg= False, eeg=False, eog=True, stim=False)

            eog_channel_names = ['EOGV', 'EOGH']
            

            # pass band
            raw_F= raw.copy()
            l_p = 150.0  # low-pass filter to suppress noise above 150 Hz
            h_p = 0.2

            raw_Filtered = raw_F.filter(l_freq=h_p, h_freq=l_p, fir_design='firwin', picks=eeg_eog_picks, filter_length='auto', method='fir', phase='zero', verbose=True)

            notches = [50, 85]
            raw_Filtered = raw_Filtered.notch_filter(notches, phase='zero-double', fir_design='firwin')
            
            # Low-frequency filter
            filt_raw_pour_ICA = raw_Filtered.copy().filter(l_freq=1.0, h_freq=None)
            
            # TODO : @Ronald : Est-ce que je fais ma dérivée sur le filt_raw_pour_ICA (dérivée qui servira ensuite à déterminer les components de l'ICA) ou n'est-ce pas nécessaire avec la dérivée ?
            # Me met toujours le warning même quand je le fait avec la dérivé !

            # Define the filter parameters
            fs = filt_raw_pour_ICA.info['sfreq']  # sampling frequency
            f0 = fs / 4  # center frequency of the filter, in Hz
            # sigma value : Doit mettre pente concordante avec vitesse. Moyenne vitesse saccades 131 ms pour part 12 à 17 TODO prendre durée moyenne saccades par participant.e.s ou en général ? Roanld réponse : engénéral
            sigma =  0.0131#(f0 / (2 * np.pi)) / 1000 # f0 / (2 * np.pi)  # standard deviation of the filter, in seconds, 
            n = 1/10 *fs #int(sigma * fs * 8) + 1  # length of the filter kernel (must be odd)

            # Create the filter kernel (a Gaussian function)
            t = np.arange(n) / fs - (n - 1) / (2 * fs)
            kernel = signal.gaussian(n, sigma * fs)

            kernel = np.diff(kernel)
            # Normalize the kernel to have unit energy
            kernel /= np.sqrt(np.sum(kernel ** 2))

            # Apply the filter to the EEG data
            raw_deriv = filt_raw_pour_ICA.copy()
            raw_deriv = raw_deriv.filter(l_freq=None, h_freq=None, method='fir', fir_window='hamming', fir_design='firwin', phase='zero-double', verbose='INFO')
            raw_deriv._data = signal.convolve(raw_deriv._data, kernel[np.newaxis, :], mode='valid')

            # Create a new Raw instance of the filtered data
            ch_types_Ronald = [mne.io.pick.channel_type(filt_raw_pour_ICA.info, n) for n in range(70)]
            info_filt = mne.create_info(ch_names=filt_raw_pour_ICA.ch_names, sfreq=filt_raw_pour_ICA.info['sfreq'], ch_types = ch_types_Ronald)
            raw_deriv = mne.io.RawArray(raw_deriv._data, info_filt) #TODO : à mettre ou pas ?
            raw_deriv.set_montage(montage, on_missing = 'warn', verbose=False)


            # Fitting ICA
            ica_pour_derivative = mne.preprocessing.ICA(n_components=0.90, method='picard', fit_params=dict(extended=False), max_iter='auto', random_state=97)  # TO DO why fit_params?
            ica_pour_derivative.fit(raw_deriv, picks = eeg_picks)  # not on raw to see better even if ica fit on filt_raw TODO : faire le fit sur les essais
            # autre filtrage meiux à fair esur bloc pour éviter effet de bord 

            ica_pour_derivative.exclude = []
            # find which ICs match the EOG pattern
            eog_indices, eog_scores = ica_pour_derivative.find_bads_eog(raw_deriv, threshold =0.7, measure = "correlation")
            ica_pour_derivative.exclude = eog_indices

            # Apply ICA to the raw data (not derivative)
            raw_after_ICA = raw_Filtered.copy()
            ica_pour_derivative.apply(raw_after_ICA)

            # Save the filtered data in BrainVision format
            path_raw_filtered = BEGIN_PATH_SAVE_FIGURE + Participant + '_RAW_AFTER_ICA.fif'
            raw_after_ICA.save(path_raw_filtered, overwrite = True)


            # Plot 

            # plot RAW Filtered before ICA
            raw_Filtered_plot= raw_Filtered.plot()
            plt.subplots_adjust(top=0.9)
            raw_Filtered_plot.suptitle("Raw filtered before ICA" + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_1_RAW_FILTERED_BEFORE_ICA'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot the filter apply to the raw_data
            filter_apply_plot = mne.viz.plot_filter(kernel, filt_raw_pour_ICA.info['sfreq'])
            plt.subplots_adjust(top=0.9)
            filter_apply_plot.suptitle("Filter apply on Raw data to derive it. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_2_FILTER_APPLIED'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot derivative of raw data before ICA application
            raw_deriv_plot = raw_deriv.plot()
            plt.subplots_adjust(top=0.9)
            raw_deriv_plot.suptitle("Derivative of Raw data before ICA application. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_3_RAW_DERIVATIVE'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # Plot the ICA sources
            source_ica_plot = ica_pour_derivative.plot_sources(raw_deriv, show_scrollbars=True, start=0, stop=20)
            plt.subplots_adjust(top=0.9)
            source_ica_plot.suptitle("Composantes ICA fig_ica_pour_derivative_EEG_channels")
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_4_ICA_DERIVATIVE_EEG'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # barplot of ICA component "EOG match" scores
            EOG_match_score_plot = ica_pour_derivative.plot_scores(eog_scores)
            plt.subplots_adjust(top=0.9)
            EOG_match_score_plot.suptitle("EOG Match scores")
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_5_ EOG_MATCH_SCORES'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot diagnostics
            diagnostics_plot = ica_pour_derivative.plot_properties(raw_deriv, picks=eog_indices)
            plt.subplots_adjust(top=0.9)
            figD = plt.gcf()
            figD.suptitle("ICA Diagnostics")
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_6_ICA_DIAGNOSTICS'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot ICs applied to raw data, with EOG matches highlighted
            ICs_plot = ica_pour_derivative.plot_sources(raw_deriv, show_scrollbars=False)
            plt.subplots_adjust(top=0.9)
            ICs_plot.suptitle("ICs")
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_7_ICs'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()

            # plot RAW AFTER ICA
            raw_after_ICA_plot= raw_after_ICA.plot()
            plt.subplots_adjust(top=0.9)
            raw_after_ICA_plot.suptitle("Raw filtered after ICA application. " + Participant)
            path_fig = BEGIN_PATH_SAVE_FIGURE + Participant + '_8_RAW_AFTER_ICA'
            plt.savefig(path_fig, transparent = False)
            # Close the figure
            plt.close()
'''


            